### Imports

In [49]:
import pandas as pd
from glob import glob
import datetime as dt
import psycopg, psycopg2

#### Load Files and look at data
We are missing values in the user type and birth also 1900 is a null for birth year 0 is a null for gender assuming 1 is male we will replace  with M, f
there is no such thing as a trip duration that last for a million seconds so we will have to look into that

In [50]:
files =glob('./data/JC-*.csv')
df_list =[]
for file in files:
    df_list.append(pd.read_csv(file))
bike_df=pd.concat(df_list)
display(bike_df.head())
display(bike_df.count())
display(bike_df.describe())
bike_df.dtypes

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0


Trip Duration              247584
Start Time                 247584
Stop Time                  247584
Start Station ID           247584
Start Station Name         247584
Start Station Latitude     247584
Start Station Longitude    247584
End Station ID             247584
End Station Name           247584
End Station Latitude       247584
End Station Longitude      247584
Bike ID                    247584
User Type                  247204
Birth Year                 228585
Gender                     247584
dtype: int64

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender
count,2.475840e+05,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000,247584.000000
mean,8.856305e+02,3207.065206,40.723121,-74.046438,3203.572553,40.722594,-74.045855,24935.260481,1979.335276,1.123534
std,3.593798e+04,26.955103,0.008199,0.011211,61.579494,0.007958,0.011283,748.469712,9.596809,0.518687
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,40.692216,-74.096937,14552.000000,1900.000000,0.000000
25%,2.480000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24491.000000,1974.000000,1.000000
50%,3.900000e+02,3201.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043117,24609.000000,1981.000000,1.000000
75%,6.660000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24719.000000,1986.000000,1.000000
max,1.632981e+07,3426.000000,40.752559,-74.032108,3426.000000,40.801343,-73.957390,27274.000000,2000.000000,2.000000


Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
dtype: object

Clean bike data making M/F replacements and breaking off the stations data to its own table converting duration into a time delta and altimitly and interval

In [51]:

bike_df['Trip Duration']=pd.to_timedelta(bike_df['Trip Duration'],'s')

bike_df['Gender']=bike_df['Gender'].astype(str)
bike_df.loc[(bike_df['Gender']==0),'Gender']=None
bike_df.loc[(bike_df['Gender']==1),'Gender']='M'
bike_df.loc[(bike_df['Gender']==2),'Gender']='F'
bike_df.loc[(bike_df['Birth Year']==1900),'Birth Year']=None

stations_df1=bike_df.loc[:,['Start Station ID','Start Station Name','Start Station Latitude','Start Station Longitude']]
stations_df1.columns=['Station_ID','Station_Name','Station_Latitude','Station_Longitude']
stations_df2=bike_df.loc[:,['End Station ID','End Station Name','End Station Latitude','End Station Longitude']]
stations_df2.columns=['Station_ID','Station_Name','Station_Latitude','Station_Longitude']
stations_df =pd.concat([stations_df1,stations_df2], axis=0).drop_duplicates()
bike_df.drop(columns=['Start Station Name','Start Station Latitude','Start Station Longitude','End Station Name','End Station Latitude','End Station Longitude'], inplace=True)
bike_df.dropna(inplace=True)
stations_df

,Station_ID,Station_Name,Station_Latitude,Station_Longitude
0,3186,Grove St PATH,40.719586,-74.043117
3,3209,Brunswick St,40.724176,-74.050656
4,3195,Sip Ave,40.730743,-74.063784
7,3211,Newark Ave,40.721525,-74.046305
8,3187,Warren St,40.721124,-74.038051
...,...,...,...,...
1400,2004,6 Ave & Broome St,40.724399,-74.004704
5142,393,E 5 St & Avenue C,40.722992,-73.979955
15834,401,Allen St & Rivington St,40.720196,-73.989978
19456,376,John St & William St,40.708621,-74.007222


In [6]:
stations_df
stations_df.dtypes

Station_ID             int64
Station_Name          object
Station_Latitude     float64
Station_Longitude    float64
dtype: object

Insert into DB

In [ ]:
with psycopg.connect("host=basement dbname=city_bike user=postgres password=****") as conn:
    cur1=conn.cursor()
    ins_sql='''
insert into bike_stations (ID, Name, Longitude, Latitude)
values(%s, %s, %s, %s)  
'''
    cur1.executemany(ins_sql,stations_df.values)
    

In [8]:


bike_df['Start Time']=pd.to_datetime(bike_df['Start Time'])
bike_df['Stop Time']=pd.to_datetime(bike_df['Stop Time'])
bike_df['Birth Year']=bike_df['Birth Year'].astype('int64')
bike_df.dtypes

Trip Duration       timedelta64[ns]
Start Time           datetime64[ns]
Stop Time            datetime64[ns]
Start Station ID              int64
End Station ID                int64
Bike ID                       int64
User Type                    object
Birth Year                    int64
Gender                       object
dtype: object

Use Copy because its a bulk insert instead of transactional

In [10]:
with psycopg.connect("host=basement dbname=city_bike user=postgres password=****") as conn:
    cursor=conn.cursor()
    with cursor.copy('COPY trips (Duration, Start_time, End_time, Start_Stn_id, End_Stn_id, Bike_id, User_type, Birth_year, Gender) FROM STDIN') as copy:
        for row in bike_df.values:
            copy.write_row(row)




Load and Clean up Weather data

In [15]:
weather_df = pd.read_csv('./data/newark_airport_2016.csv')

weather_df['airport']=weather_df['NAME'].str.split(',',expand=True)[0]
state_country=weather_df['NAME'].str.split(',', expand=True)[1]
weather_df['state']=state_country.str[1:3]
weather_df['country']=state_country.str[4:]
weather_df['DATE']=weather_df['DATE'].astype('datetime64[s]')
weather_df['AWND_dir']= weather_df[['WDF2','WDF5',]].mean(axis=1)
weather_df.drop(columns=['PGTM', 'TSUN','NAME', 'WSF2', 'WSF5', 'WDF2','WDF5'], inplace=True)
weather_df.rename(columns={'SNOWD':'SNOW_depth', 'TMAX':'Temp_max', 'TMIN':'Temp_min', 'TAVG':'Temp_avg'}, inplace=True)
weather_df

,STATION,DATE,AWND,PRCP,SNOW,SNWD,Temp_avg,Temp_max,Temp_min,airport,state,country,AWND_dir
0,USW00014734,2016-01-01,12.75,0.00,0.0,0.0,41,43,34,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,275.0
1,USW00014734,2016-01-02,9.40,0.00,0.0,0.0,36,42,30,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,260.0
2,USW00014734,2016-01-03,10.29,0.00,0.0,0.0,37,47,28,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,260.0
3,USW00014734,2016-01-04,17.22,0.00,0.0,0.0,32,35,14,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,330.0
4,USW00014734,2016-01-05,9.84,0.00,0.0,0.0,19,31,10,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,355.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,USW00014734,2016-12-27,13.65,0.01,0.0,0.0,53,62,40,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,270.0
362,USW00014734,2016-12-28,8.28,0.00,0.0,0.0,41,43,31,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,330.0
363,USW00014734,2016-12-29,8.05,0.36,0.0,0.0,38,45,31,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,160.0
364,USW00014734,2016-12-30,14.99,0.00,0.0,0.0,37,42,32,NEWARK LIBERTY INTERNATIONAL AIRPORT,NJ,US,270.0


insert into DB there are acutally 2 tables here the site informantion and the weather at that site but we only have 1 site of data

In [43]:
with psycopg2.connect("host=basement dbname=city_bike user=postgres password=****") as conn:
    cur1=conn.cursor()
    ins_sql='''
insert into weather_station (ID, names, state)
values(%s, %s, %s)  
'''
    cur1.executemany(ins_sql,weather_df[['STATION', 'airport', 'state']].drop_duplicates().values)

    ins_sql='''insert into weather_repts (w_station, rep_date, avg_wind_spd, precipitation, snow, snow_depth, temp_avg, temp_max,temp_min)
    values(%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
    cur1.executemany(ins_sql, weather_df[['STATION', 'DATE', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'Temp_avg','Temp_max', 'Temp_min']].values)